In [ ]:
import importlib
import numpy as np
import matplotlib.pyplot as plt
import configparser
import copy
import csv
import pathlib

import colors, plot_func, helpers, file_helpers, experiments

colors_lst, _, _ = colors.color_scheme()

## Supervisor

In [ ]:
importlib.reload(file_helpers)
importlib.reload(helpers)

import SupervisorClass
importlib.reload(SupervisorClass)
from SupervisorClass import SupervisorClass

Sprvsr = SupervisorClass()
# Sprvsr.init_dataset(Sprvsr.dataset_path)

## Sensor

In [ ]:
importlib.reload(file_helpers)
importlib.reload(plot_func)

import ForsentekClass
importlib.reload(ForsentekClass)
from ForsentekClass import ForsentekClass

Snsr = ForsentekClass()

## robot

In [ ]:
if "m" in globals():
    try:
        m.disconnect()
    except Exception as e:
        print("Robot was not connected or already disconnected:", e)
    del m


import MecaClass
importlib.reload(MecaClass)
importlib.reload(file_helpers)
importlib.reload(helpers)
from MecaClass import MecaClass

m = MecaClass(Sprvsr)

In [ ]:
m.connect()
# m.move_to_origin()
m._get_current_pos()
print(f'current pos {m.current_pos}')
print(f'relative to origin {m.current_pos[:2]-m.pos_origin}')

In [ ]:
m.set_frames('training')
# m.robot.MoveLin(129.5, -12.5, 31, -180, 0, 0)  # up
m.move_pos_w_mid([128, 2, 110], Sprvsr)
m.current_pos

## Calibrate

In [ ]:
# Without chain you can use False
# If with chain, use True

Snsr.calibrate_daily(V0_from_file = False)

## Predetermined trajectory

In [ ]:
## Measure
importlib.reload(file_helpers)
importlib.reload(helpers)
importlib.reload(experiments)

# around Jan 20
# x_range = 240
# y_range = 85
# theta_range = 45
# Jan 22
# x_range = 0.25*1000 + 10
# y_range = 0.02*1000 - 14
# theta_range = 0.4*180/np.pi
# Jan 23 metal shims
# x_range = 160 + m.pos_origin[0]
# y_range = 80 + m.pos_origin[1]
# theta_range = np.rad2deg(-0.5)
# N = 1
# Feb 16
path = 'C:/Users/owner/OneDrive - huji.ac.il/PhD/Research/Bistable shape acquisition jax/Predetermined trajectory/Feb17/x=160/traj_x=160_y=-71to99_angle=15.csv'
x_y_theta_vec, F_vec = experiments.sweep_measurement_fixed_origami(m, Snsr, 
                                                                   Sprvsr,
                                                                   path=path)                                                      
print('x_y_theta_vec', x_y_theta_vec)
print('F_vec', F_vec)

In [ ]:
y_pos = x_y_theta_vec[:, 1]
plt.errorbar(y_pos, F_vec[:, 0], yerr=0.01*1e3)
plt.errorbar(y_pos, F_vec[:, 1], yerr=0.01*1e3)
plt.ylim([-100, 160])

In [ ]:
F_vec

In [ ]:
Snsr.measure(0.5)
Sprvsr.global_force(Snsr, m, plot=True)
print('Fx', Sprvsr.Fx)
print('Fy', Sprvsr.Fy)

In [ ]:
importlib.reload(file_helpers)

# path = 'C:/Users/owner/OneDrive - huji.ac.il/PhD/Research/Bistable shape acquisition jax/Predetermined trajectory/Feb17/x=160/buckle=0010/forces_x=160_buckle=0010_angle=15.csv'
path_straight = 'data/measurements/Feb25/x=160_buckle=1001.csv'
# path_flipped = 'data/measurements/Feb22/Predetermined traj, buckle=0001/x=160_buckle=1101_flipped_2.csv'
path_sim_1 = 'C:/Users/owner/OneDrive - huji.ac.il/PhD/Research/Bistable shape acquisition jax/Predetermined trajectory/Feb23/1001/L=0.045_buckle[ 1 -1 -1  1].csv'
path_sim_2 = 'C:/Users/owner/OneDrive - huji.ac.il/PhD/Research/Bistable shape acquisition jax/Predetermined trajectory/Feb24/1001/L=0.045_buckle[ 1 -1 -1  1].csv'
# path_straight = 'data/measurements/Feb22/Predetermined traj, buckle=0001/x=160_buckle=0001_straight.csv'
# path_flipped = 'data/measurements/Feb22/Predetermined traj, buckle=0001/x=160_buckle=0001_flipped.csv'
# path_old = 'data/measurements/Feb17/x=160_buckle=0001/x=160_buckle=0001.csv'
# path_sim = 'C:/Users/owner/OneDrive - huji.ac.il/PhD/Research/Bistable shape acquisition jax/Predetermined trajectory/Feb23/0001/L=0.045_buckle[-1 -1 -1  1].csv'

rows = file_helpers.load_pos_force(path_straight)  # uses your CSV format
x_y_theta_vec_straight = np.array([r["pos"] for r in rows], dtype=float)
F_vec_straight = np.array([r["force"] for r in rows], dtype=float)

# rows = file_helpers.load_pos_force(path_flipped)  # uses your CSV format
# x_y_theta_vec_flipped = np.array([r["pos"] for r in rows], dtype=float)
# F_vec_flipped = np.array([r["force"] for r in rows], dtype=float)

# rows = file_helpers.load_pos_force(path_old)  # uses your CSV format
# x_y_theta_vec_old = np.array([r["pos"] for r in rows], dtype=float)
# F_vec_old = np.array([r["force"] for r in rows], dtype=float)
# F_vec_old = F_vec_old * 1e3

rows = file_helpers.load_pos_force(path_sim_1)  # uses your CSV format
x_y_theta_vec_sim_1 = np.array([r["pos"] for r in rows], dtype=float)
F_vec_sim_1 = np.array([r["force"] for r in rows], dtype=float)

rows = file_helpers.load_pos_force(path_sim_2)  # uses your CSV format
x_y_theta_vec_sim_2 = np.array([r["pos"] for r in rows], dtype=float)
F_vec_sim_2 = np.array([r["force"] for r in rows], dtype=float)

y_pos = x_y_theta_vec_straight[:, 1]
plt.errorbar(y_pos, F_vec_straight[:, 0], yerr=0.01*1e3, color = colors_lst[0])
plt.errorbar(y_pos, F_vec_straight[:, 1], yerr=0.01*1e3, color = colors_lst[1])
# plt.errorbar(y_pos, F_vec_flipped[:, 0], yerr=0.01*1e3, color = colors_lst[0],
#             linestyle='--')
# plt.errorbar(y_pos, F_vec_flipped[:, 1], yerr=0.01*1e3, color = colors_lst[1],
#             linestyle='--')
# plt.errorbar(y_pos, F_vec_old[:, 0], yerr=0.01*1e3, color = colors_lst[0],
#             linestyle='-.')
# plt.errorbar(y_pos, F_vec_old[:, 1], yerr=0.01*1e3, color = colors_lst[1],
#             linestyle='-.')
plt.errorbar(y_pos, F_vec_sim_1[:, 0], color = colors_lst[2],
            linestyle='-')
plt.errorbar(y_pos, F_vec_sim_1[:, 1], color = colors_lst[2],
            linestyle='-')
plt.errorbar(y_pos, F_vec_sim_2[:, 0], color = colors_lst[3],
            linestyle='--')
plt.errorbar(y_pos, F_vec_sim_2[:, 1], color = colors_lst[3],
            linestyle='--')
plt.ylabel('F [mN]')
# plt.ylim([-0.015, 0.085])
plt.xlabel('y [mm]')
# plt.legend([ r'$F_{x,straigth}$', r'$F_{y,straigth}$', r'$F_{x,flipped}$', 
#              r'$F_{y,flipped}$', r'$F_{x,old}$', r'$F_{y,old}$'])
# plt.legend([ r'$F_{x,straight}$', r'', r'$F_{x,flipped}$', 
#              r'', r'$F_{x,old}$', r'', r'$F_{x,sim}$'])
ax = plt.gca()
ax.legend(
    [r'$F_{x,straight}$', r'$F_{y,straight}$', 
     # r'$F_{x,flipped}$', r'$F_{y,flipped}$', 
     # r'$F_{x,old}$', r'$F_{y,old}$', 
     r'$F_{x,sim1}$', r'$F_{y,sim1}$',
     r'$F_{x,sim2}$', r'$F_{y,sim2}$'],
    loc='center left',
    bbox_to_anchor=(1.02, 0.5),
    frameon=False
)

plt.tight_layout()
plt.show()

In [ ]:
F_vec_straight[:,0]

In [ ]:
F_tot = np.sqrt(F_vec[0,:]**2 + F_vec[1,:]**2)
err = np.sqrt((0.01*1e3)**2 + (0.01*1e3)**2)
F_tot_sim = np.sqrt(F_vec_sim[:,0]**2 + F_vec_sim[:,1]**2)
plt.errorbar(y_pos, F_tot*1e3, yerr=err, color = colors_lst[0])
plt.plot(y_pos, F_tot_sim, color = colors_lst[0])
plt.ylabel('F [mN]')
# plt.ylim([-0.015, 0.085])
plt.xlabel('y [mm]')
plt.legend([r'$||F||$', r'$||F||_{sim}$'])

In [ ]:
## Save as csv

importlib.reload(file_helpers)
# origin = np.array([154-30, -14, 0])  # 5piece plastic shims
# origin = m.pos_origin
# origin = np.append(origin, 0)
origin = np.array([0, 0, 0])  # for when using sim data
x_y_theta_vec_moved = x_y_theta_vec - origin
print(x_y_theta_vec_moved)
out_path = "x=160_buckle=0010.csv"
file_helpers.write_supervisor_dataset(x_y_theta_vec_moved, F_vec, out_path=out_path)

## Training

In [ ]:
## Load dataset

import SupervisorClass
importlib.reload(SupervisorClass)
from SupervisorClass import SupervisorClass
importlib.reload(experiments)

measure_des = True
# measure_des = False

importlib.reload(file_helpers)

# Sprvsr.init_dataset(dataset_path = "data/datasets/Jan29/buckle[0111]minus.csv")
print('position chain in desired buckle')
Sprvsr.init_dataset(dataset_path = Sprvsr.dataset_path, measure_des = measure_des, 
                    m=m, Snsr=Snsr)
print('dataset=', Sprvsr.pos_in_t)
print('position chain in initial buckle')

In [ ]:
## Train
for t in range(Sprvsr.T):

    # # measurement
    print('tip position = ', Sprvsr.pos_in_t[t, :])
    m.move_pos_w_mid(Sprvsr.pos_in_t[t, :], Sprvsr)
    Snsr.measure()
    Sprvsr.global_force(Snsr, m, t, plot=True)
    print('measurement force = ', [Sprvsr.Fx, Sprvsr.Fy])
    
    Sprvsr.calc_loss(t, Snsr.norm_force)
    print('loss (normalized) = ', Sprvsr.loss_norm)

    # # update
    if Sprvsr.experiment == 'training':
        Sprvsr.calc_tip_update(m, t)
    elif Sprvsr.experiment == 'predetermined training':
        pass
    m.move_pos_w_mid(Sprvsr.pos_update_in_t[t, :], Sprvsr)

In [ ]:
## Print out results of training

print('update pos in t', Sprvsr.pos_update_in_t)
print('Sprvsr.F_in_t', Sprvsr.F_in_t)
print('Sprvsr.desired_F_in_t', Sprvsr.desired_F_in_t)
plt.plot(range(Sprvsr.T), Sprvsr.loss_norm_in_t)

In [ ]:
Sprvsr.desired_F_in_t

In [ ]:
## Save as csv

importlib.reload(file_helpers)

T = Sprvsr.T-1

# path_csv = f"{np.array2string(Sprvsr.init_buckle)}to{np.array2string(Sprvsr.desired_buckle)}.csv"
path_csv = f"0001to1000.csv"
file_helpers.export_training_csv(path_csv, Sprvsr, T=T)

In [ ]:
## plot important training results

importlib.reload(plot_func)
final_t = 7

# file_path = "data/measurements/Feb18/0001to1000/0001to1000.csv" 
file_path = "data/measurements/Feb19/0001to1000_2_success/0001to1000.csv" 
plot_func.importants_from_file(file_path, final_t=final_t, save=True)

## Stress Strain

In [ ]:
m.set_frames(mod='stress_strain')
# m.robot.MoveLin(129.50, -12.5, 31, -180, 0, 0)  # down
# m.robot.MoveLin(129.5, -12.5, 61, -180, 0, 0)  # up
m.move_pos_w_mid([Sprvsr.L, 0 , 0], Sprvsr)

In [ ]:
importlib.reload(experiments)

theta_range = 135
theta_ss = 60
N = 100
y_step = 55
x_step = 15
connect_hinge = False
move_mod = "lin"
sweep_mod = "pole"

thetas_vec, Fx_vec, Fy_vec = experiments.stress_strain(m, Snsr, 
                                                       theta_range,
                                                       theta_ss, N, 
                                                       y_step, x_step,
                                                       connect_hinge = connect_hinge, 
                                                       move_mod = move_mod, 
                                                       sweep_mod = sweep_mod)


In [ ]:
scale = 1000  # mili
arm_len = 0.0225

torque_x = Fx_vec * arm_len * scale
torque_y = Fy_vec * arm_len * scale
inds = torque_x<0

# plt.plot(np.deg2rad(thetas_vec[inds]), torque_x[inds])
# plt.plot(np.deg2rad(thetas_vec[inds]), torque_y[inds])
plt.plot(np.deg2rad(thetas_vec), -torque_x)
plt.plot(np.deg2rad(thetas_vec), -torque_y)
# plt.plot(np.deg2rad(thetas_vec), Fx_vec)
# plt.plot(np.deg2rad(thetas_vec), Fy_vec)
plt.xlabel(r'$\theta\,\left[rad\right]$')
plt.legend([r'$\tau_x$', r'$\tau_y$'])
plt.ylim([-15, 15])
plt.xlim([-np.pi, np.pi])
plt.ylabel(r'$\tau$')
plt.show()

In [ ]:


out_path = 'Stress_Strain_1myl1tp_otherEnd.csv'
file_helpers.save_stress_strain(out_path, thetas_vec, Fx_vec, Fy_vec, torque_x,
                                torque_y)

# out_path = Path(out_path)
# mode = "w"
# write_header = (mode == "w") or (not out_path.exists())

# with out_path.open(mode, newline="", encoding="utf-8") as f:
#     writer = csv.writer(f)

#     if write_header:
#         writer.writerow(["theta", "F_x", "F_y", "torque_x", "torque_y"])

#     for i in range(thetas_vec.shape[0]):
#         writer.writerow([thetas_vec[i], Fx_vec[i], Fy_vec[i], 
#                          torque_x[i], torque_y[i]])
#         f.flush()

In [ ]:
importlib.reload(file_helpers)

path = "C:/Users/owner/OneDrive - huji.ac.il/PhD/Research/Robotic arm/Meca500/data/Stress Strain/19Feb/Stress_Strain_1myl1tp_otherEnd.csv"
Leon_path = "C:/Users/owner/OneDrive - huji.ac.il/PhD/Research/Robotic arm/Meca500/data/Stress Strain/Leon's/Roee_metal_offset3mm_dl75.txt"
noShm_path = "C:/Users/owner/OneDrive - huji.ac.il/PhD/Research/Robotic arm/Meca500/data/Stress Strain/15Feb/1Myl1tp_noShm.csv"
onlyWires_path = "C:/Users/owner/OneDrive - huji.ac.il/PhD/Research/Robotic arm/Meca500/data/Stress Strain/15Feb/onlyWires.csv"
thetas_vec, Fx_vec, Fy_vec, torque_x, torque_y = file_helpers.load_stress_strain(path, file_type='csv')
Leon_thetas_vec, _, _, Leon_torque_x, _ = file_helpers.load_stress_strain(Leon_path, file_type='txt')
noShm_thetas_vec, _, _, noShm_torque_x, noShm_torque_y = file_helpers.load_stress_strain(noShm_path, file_type='csv')
onlyWires_thetas_vec, _, _, onlyWires_torque_x, onlyWires_torque_y = file_helpers.load_stress_strain(onlyWires_path, file_type='csv')

plt.plot(np.deg2rad(thetas_vec), -torque_x, color=colors_lst[0])
plt.plot(np.deg2rad(thetas_vec), -torque_y, color=colors_lst[1])
# plt.plot(np.deg2rad(noShm_thetas_vec), -noShm_torque_x, color=colors_lst[0])
# plt.plot(np.deg2rad(noShm_thetas_vec), -noShm_torque_y, color=colors_lst[1])
plt.plot(np.deg2rad(-Leon_thetas_vec), -Leon_torque_x, color=colors_lst[2])
plt.plot(np.deg2rad(Leon_thetas_vec), Leon_torque_x, color=colors_lst[2])
plt.xlabel(r'$\theta\,\left[rad\right]$')
plt.legend([r'$\tau_x$', r'$\tau_y$', 'Leons'])
plt.ylim([-15, 15])
plt.xlim([-np.pi, np.pi])
plt.show()

# plt.plot(np.deg2rad(thetas_vec), -torque_x - (-noShm_torque_x), color=colors_lst[0])
# plt.plot(np.deg2rad(-Leon_thetas_vec), -Leon_torque_x, color=colors_lst[2])
# plt.plot(np.deg2rad(Leon_thetas_vec), Leon_torque_x, color=colors_lst[2])
# plt.xlabel(r'$\theta\,\left[rad\right]$')
# plt.legend([r'$\tau_x$','Leons'])
# plt.ylim([-15, 15])
# plt.xlim([-np.pi, np.pi])
# plt.show()

# plt.plot(np.deg2rad(onlyWires_thetas_vec), -onlyWires_torque_x, color=colors_lst[0])
# plt.plot(np.deg2rad(onlyWires_thetas_vec), -onlyWires_torque_y, color=colors_lst[1])
# plt.plot(np.deg2rad(noShm_thetas_vec), -noShm_torque_x, color=colors_lst[0])
# plt.plot(np.deg2rad(noShm_thetas_vec), -noShm_torque_y, color=colors_lst[1])
# plt.xlabel(r'$\theta\,\left[rad\right]$')
# plt.legend([r'no shim x', r'no shim y', 'only wires x', 'only wires y'])
# plt.ylim([-15, 15])
# plt.xlim([-np.pi, np.pi])
# plt.show()


In [ ]:
plt.plot(np.deg2rad(thetas_vec), -Fx_vec)
plt.plot(np.deg2rad(thetas_vec), -Fy_vec)
plt.xlabel(r'$\theta\,\left[rad\right]$')
plt.legend([r'$F_x$', r'$F_y$'])
plt.ylim([-1, 1])
plt.show()

## Check Robot movement

In [ ]:
# m.robot.MoveLin(140, 0, 180, 180, 0, 0)
# points = (220.0, 0.0, 360, 0, 0, 90)
points = (250, 55, -73)  # training metal shims 4piece
# points = (170,  135, -190)
# points = (115, -14, 0)
m.move_pos_w_mid(points, Sprvsr)
# joints = (0, 60, 30, -90, -90, 180)
# m.move_joints(joints)

In [ ]:
m._get_current_pos()
m.current_pos
# m.current_pos[-1]

In [ ]:
importlib.reload(plot_func)
# m.robot.MoveLin(129.50, -12.5, 31, -180, 0, 10)  # down 12Feb
# m.robot.WaitIdle()
# pos1=([160,         -22,  15        ])
# pos2=([160,           79.8,  15        ])
# m.move_pos_w_mid(pos2, Sprvsr)
# joints = np.array([[0, 60, 30, -90, -90, 90],  # x axis force sensor at ai1
#                    [0, 60, 30, -90, -90, 180],   # y axis force sensor at ai2
#                    [0, 60, -60, 0, -90, 180]])
# m.move_joints(joints[1, :])
Snsr.measure(0.5)
Sprvsr.global_force(Snsr, m, plot=True)
print('Fx', Sprvsr.Fx)
print('Fy', Sprvsr.Fy)

In [ ]:
len(Snsr.force_data[:,0])

In [ ]:
mean_afo_len = np.zeros(len(Snsr.force_data[:,0]))
std_afo_len = np.zeros(len(Snsr.force_data[:,0]))
for i in range(len(mean_afo_len)):
    mean_afo_len[i] = np.mean(Snsr.force_data[:i,0])    
    std_afo_len[i] = np.std(Snsr.force_data[:i,0])

plt.plot(range(len(mean_afo_len)), mean_afo_len)
plt.show()
plt.plot(range(len(mean_afo_len)), std_afo_len)
plt.show()


## Predetermined trajectory

## Detemined set

In [ ]:
# Sprvsr.pos_in_t = np.array([[180, 40, 0], [180, 40, -40], [180, 40, -80], 
#                             [180, 40, -120], [180, 40, -140], [180, 40, -160], 
#                             [180, 40, -180], [180, 40, -200] ])
# Sprvsr.pos_in_t = np.array([[180, 70, -90]])
# Sprvsr.pos_in_t = np.array([[180, 70, -90], [180, 75, -90], [180, 80, -90], 
#                             [180, 85, -90], [180, 90, -90], [180, 95, -90], 
#                             [180, 100, -90], [180, 105, -90]])
# Sprvsr.T = 8
# tri = 1 - np.abs(2 * (np.arange(Sprvsr.T) / (Sprvsr.T - 1)) - 1)   # 0 → 1 → 0
# x_pos = 230 + 10 * tri                   # 170 → 210 → 170
# y_pos = np.linspace(-50, 50, Sprvsr.T)
# Sprvsr.pos_in_t = np.zeros([Sprvsr.T, 3])
# Sprvsr.F_in_t = np.zeros([Sprvsr.T, 2])

# for t in range(Sprvsr.T):
#     Sprvsr.pos = np.array([x_pos[t], y_pos[t], 0])
#     Sprvsr.pos_in_t[t, :] = Sprvsr.pos
#     print('tip position = ', Sprvsr.pos)
#     m.move_pos_w_mid(Sprvsr.pos, Sprvsr)
#     Snsr.measure()
#     Sprvsr.global_force(Snsr, m, plot=True)
#     plt.show()
#     Sprvsr.F_in_t[t, :] = np.array([Sprvsr.Fx, Sprvsr.Fy])
#     print('mean force = ', [Sprvsr.Fx, Sprvsr.Fy])



In [ ]:
m.disconnect()

## Logging - fixed arm

In [ ]:
experiments.sweep_measurement_fixed_origami(x_range, y_range, theta_range, N, robot, force_sensor)

## Calibrate forces

In [ ]:
importlib.reload(experiments)

weights_gr = [2, 5, 10, 20, 50]
experiments.calibrate_forces_all_axes(m, Snsr, weights_gr)

## Recover robot after error

In [ ]:
m._recover_robot()

## End of day

In [ ]:
# m.robot.MoveLin(40, 0, 230, 180, 0, 0)
m.move_to_sleep_pos()

In [ ]:
m.disconnect()